In [ ]:
pip install cairosvg

In [ ]:
import requests
import cairosvg
from PIL import Image, ImageSequence
import io
import os
from datetime import datetime

def download_svg(url):
    """Download SVG from URL"""
    response = requests.get(url)
    response.raise_for_status()  # Check if the request was successful
    return response.content

def convert_svg_to_gif(svg_content, output_path, delay=100):
    """
    Convert SVG to GIF by splitting it into frames
    """
    try:
        # Convert SVG to PNG bytes
        png_bytes = cairosvg.svg2png(bytestring=svg_content)
        
        # Open PNG as PIL Image
        img = Image.open(io.BytesIO(png_bytes))
        
        # Get image dimensions
        width, height = img.size
        
        # Each segment is typically 111px wide for GitHub snake
        segment_width = 111
        num_frames = width // segment_width
        
        # Create frames list
        frames = []
        
        for i in range(num_frames):
            # Calculate crop box for each segment
            left = width - ((i + 1) * segment_width)
            right = width - (i * segment_width)
            
            # Ensure we don't go out of bounds
            if left < 0:
                left = 0
            if right > width:
                right = width
            
            # Crop the segment
            segment = img.crop((left, 0, right, height))
            
            # Create a new image with white background
            frame = Image.new('RGB', (segment_width, height), 'white')
            
            # Paste the cropped segment
            # If the segment is smaller than segment_width (for last frame)
            segment_width_actual = segment.width
            frame.paste(segment, (segment_width - segment_width_actual, 0))
            
            frames.append(frame)
        
        # Save as GIF
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=delay,
            loop=0,  # 0 means infinite loop
            disposal=2  # Restore to background color
        )
        
        print(f"GIF saved to: {output_path}")
        print(f"Frames created: {len(frames)}")
        print(f"Delay per frame: {delay}ms")
        
    except Exception as e:
        print(f"Error converting SVG to GIF: {e}")
        raise

def convert_svg_to_gif_simple(svg_content, output_path, delay=100):
    """
    Alternative simpler method - creates sliding animation
    """
    try:
        # Convert SVG to PNG
        png_bytes = cairosvg.svg2png(bytestring=svg_content)
        img = Image.open(io.BytesIO(png_bytes))
        
        width, height = img.size
        segment_width = 111  # GitHub snake segment width
        
        # Create frames for sliding animation
        frames = []
        steps = 20  # Number of intermediate steps between segments
        
        for i in range(0, width - segment_width, segment_width // steps):
            # Crop moving window
            right = width - i
            left = right - segment_width
            
            if left < 0:
                left = 0
                right = segment_width
            
            frame = img.crop((left, 0, right, height))
            frames.append(frame)
        
        # Save as GIF
        frames[0].save(
            output_path,
            save_all=True,
            append_images=frames[1:],
            duration=delay,
            loop=0,
            optimize=True
        )
        
        print(f"GIF saved to: {output_path}")
        print(f"Total frames: {len(frames)}")
        
    except Exception as e:
        print(f"Error in simple conversion: {e}")
        raise


In [ ]:

def main():
    # Configuration
    svg_url = "https://raw.githubusercontent.com/7oSkaaa/7oSkaaa/d06ca6876dcb870062cc423d013316bd60d7e1af/github-contribution-grid-snake.svg"
    output_filename = "github_snake_animation.gif"
    
    print("Starting SVG to GIF conversion...")
    print(f"Downloading from: {svg_url}")
    
    try:
        # Step 1: Download SVG
        svg_content = download_svg(svg_url)
        print("✓ SVG downloaded successfully")
        
        # Step 2: Convert to GIF using first method (segment-based)
        print("Converting to GIF (method 1: segment animation)...")
        convert_svg_to_gif(svg_content, output_filename, delay=150)
        
        # Optional: Try the second method too
        # print("\nConverting to GIF (method 2: sliding animation)...")
        # convert_svg_to_gif_simple(svg_content, "github_snake_sliding.gif", delay=50)
        
        print("\nConversion complete!")
        
        # Display some info about the created GIF
        if os.path.exists(output_filename):
            gif = Image.open(output_filename)
            print(f"\nGIF Information:")
            print(f"  Size: {gif.size}")
            print(f"  Mode: {gif.mode}")
            print(f"  Frames: {gif.n_frames if hasattr(gif, 'n_frames') else 'Unknown'}")
            print(f"  File size: {os.path.getsize(output_filename) / 1024:.2f} KB")
        
    except requests.exceptions.RequestException as e:
        print(f"Error downloading SVG: {e}")
    except Exception as e:
        print(f"Error during conversion: {e}")

if __name__ == "__main__":
    # Install required packages if not installed
    print("Checking dependencies...")
    try:
        import cairosvg
        import PIL
    except ImportError:
        print("Installing required packages...")
        import subprocess
        import sys
        
        packages = [
            "cairosvg",
            "pillow",
            "requests"
        ]
        
        for package in packages:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
    main()

In [4]:
# Install: pip install cairosvg pillow
import cairosvg
from PIL import Image
import os
import io

def svg_to_gif(input_svg, output_gif="output.gif", frame_duration=100):
    """
    Convert local SVG to animated GIF
    
    Args:
        input_svg: Path to your SVG file
        output_gif: Output GIF filename
        frame_duration: Time per frame in milliseconds
    """
    try:
        # Read SVG file
        with open(input_svg, 'rb') as f:
            svg_data = f.read()
        
        # Convert SVG to PNG
        png_data = cairosvg.svg2png(bytestring=svg_data)
        
        # Open as PIL Image
        with Image.open(io.BytesIO(png_data)) as img:
            width, height = img.size
            
            # Create animation frames (scrolling effect)
            frames = []
            scroll_step = 20  # Pixels to move each frame
            
            for i in range(0, width, scroll_step):
                # Define crop area for scrolling
                left = max(0, width - i - 400)  # Show 400px width
                right = min(width, left + 400)
                
                if left >= right:
                    continue
                    
                # Crop frame
                frame = img.crop((left, 0, right, height))
                
                # Resize if needed for consistent frame size
                if frame.width < 400:
                    # Create new frame with background
                    new_frame = Image.new('RGB', (400, height), (255, 255, 255))
                    new_frame.paste(frame, (400 - frame.width, 0))
                    frame = new_frame
                
                frames.append(frame)
        
        # Save as animated GIF
        if frames:
            frames[0].save(
                output_gif,
                save_all=True,
                append_images=frames[1:],
                duration=frame_duration,
                loop=0,
                optimize=True
            )
            print(f"✓ GIF saved: {output_gif}")
            print(f"  Frames: {len(frames)}")
            print(f"  Size: {os.path.getsize(output_gif) / 1024:.1f} KB")
        
    except Exception as e:
        print(f"Error: {e}")

# Usage
svg_to_gif("./github-contribution-grid-snake.svg", "snake_animation.gif", frame_duration=150)

✓ GIF saved: snake_animation.gif
  Frames: 44
  Size: 73.5 KB


In [5]:
import cairosvg
import numpy as np
from moviepy.video.io.ImageSequenceClip import ImageSequenceClip
from PIL import Image
import io

# 1. Convert SVG to PNG with specific size
svg_content = open('./github-contribution-grid-snake.svg').read()
png_data = cairosvg.svg2png(
    bytestring=svg_content.encode(),
    output_width=1280,  # Set explicit size
    output_height=720
)

# 2. Convert PNG to numpy array with proper RGB conversion
img = Image.open(io.BytesIO(png_data))

# Remove alpha channel if exists (convert RGBA to RGB)
if img.mode == 'RGBA':
    # Create white background and composite
    rgb_img = Image.new('RGB', img.size, (255, 255, 255))
    rgb_img.paste(img, mask=img.split()[3])  # Use alpha channel as mask
    img = rgb_img

frame = np.array(img)

# 3. Create video (repeat same frame for duration)
clip = ImageSequenceClip([frame] * 60, fps=30)

# Write with proper settings
clip.write_videofile(
    'output.mp4',
    codec='libx264',
    fps=30,
    audio=False,
    verbose=True
)

print("Video created successfully!")

TypeError: got an unexpected keyword argument 'verbose'

In [7]:
pip install moviepy imageio imageio[ffmpeg] numpy


[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [7]:
import cairosvg
import cv2
import numpy as np
from PIL import Image
import io

# Convert SVG to PNG
svg_content = open('./github-contribution-grid-snake.svg').read()
png_data = cairosvg.svg2png(
    bytestring=svg_content.encode(),
    output_width=1280,
    output_height=720
)

# Convert to numpy array
nparr = np.frombuffer(png_data, np.uint8)
img = cv2.imdecode(nparr, cv2.IMREAD_UNCHANGED)

# Handle transparency
if img.shape[2] == 4:  # Has alpha channel
    # Split into color and alpha
    color = img[:, :, :3]
    alpha = img[:, :, 3] / 255.0
    
    # Create white background
    result = np.ones_like(color) * 255
    # Blend
    for c in range(3):
        result[:, :, c] = color[:, :, c] * alpha + 255 * (1 - alpha)
    img = result.astype(np.uint8)

# Create video
height, width = img.shape[:2]
fps = 30
duration = 2  # seconds
frame_count = fps * duration

# Create video writer
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or 'avc1' for H.264
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# Write frames
for _ in range(frame_count):
    out.write(img)

out.release()
cv2.destroyAllWindows()
print(f"Video created: {width}x{height}, {fps}fps, {duration}s")

Video created: 1280x720, 30fps, 2s


In [10]:
import cairosvg
import numpy as np
import imageio.v2 as iio
from PIL import Image
import io
import os

# 1. Convert SVG to PNG
print("Reading SVG...")
with open('./github-contribution-grid-snake.svg', 'r') as f:
    svg_content = f.read()

print("Converting SVG to PNG...")
png_data = cairosvg.svg2png(
    bytestring=svg_content.encode(),
    output_width=1920,
    output_height=1080,
    background_color='white'  # Force white background
)

# 2. Save PNG for debugging
print("Saving debug PNG...")
with open('debug_output.png', 'wb') as f:
    f.write(png_data)

# 3. Load and verify the image
img = Image.open(io.BytesIO(png_data))
print(f"Image mode: {img.mode}")
print(f"Image size: {img.size}")
print(f"Image data range: {np.array(img).min()} to {np.array(img).max()}")

# Convert to RGB if needed
if img.mode != 'RGB':
    print(f"Converting from {img.mode} to RGB...")
    img = img.convert('RGB')

# Convert to numpy array
frame = np.array(img)
print(f"Frame shape: {frame.shape}")
print(f"Frame dtype: {frame.dtype}")

# 4. Create video with proper settings
print("\nCreating video...")
fps = 30
duration = 5  # seconds
total_frames = fps * duration

# Create list of frames
frames = [frame] * total_frames

# Write video with explicit settings
try:
    # Try with proper writer
    writer = iio.get_writer(
        'output.mp4',
        fps=fps,
        codec='libx264',
        quality=8,  # 0-10, higher is better
        pixelformat='yuv420p',
        macro_block_size=16  # Important for dimensions
    )
    
    for i, frame in enumerate(frames):
        writer.append_data(frame)
        if i % 30 == 0:
            print(f"Added frame {i+1}/{total_frames}")
    
    writer.close()
    print("✅ Video created successfully!")
    
except Exception as e:
    print(f"Error with writer: {e}")
    print("Trying alternative method...")
    
    # Alternative: Use mimsave
    iio.mimsave(
        'output.mp4',
        frames,
        fps=fps,
        codec='libx264',
        quality=8
    )
    print("✅ Video created with alternative method!")

# 5. Verify the output
if os.path.exists('output.mp4'):
    file_size = os.path.getsize('output.mp4')
    print(f"\n✅ Video created: output.mp4")
    print(f"📊 File size: {file_size / 1024:.1f} KB")
    
    # Check if file has reasonable size
    if file_size < 10 * 1024:  # Less than 10KB is suspicious
        print("⚠️  Warning: File is very small. The video might be blank.")
        print("   Common issues:")
        print("   1. Wrong color space")
        print("   2. Codec not installed")
        print("   3. All frames are identical and compressed heavily")
        
        # Try creating a simple test video
        print("\n🎬 Creating test video with colored frame...")
        test_frame = np.zeros((1080, 1920, 3), dtype=np.uint8)
        test_frame[:, :] = [255, 0, 0]  # Red frame
        iio.mimsave('test_red.mp4', [test_frame] * 60, fps=30)
        print("✅ Test video created. Check if 'test_red.mp4' is red.")
else:
    print("❌ Error: Video file was not created")

print("\nDebug steps:")
print("1. Open 'debug_output.png' - does it show your image?")
print("2. Open 'output.mp4' in a video player")
print("3. If still blank, try VLC media player")

Reading SVG...
Converting SVG to PNG...
Saving debug PNG...
Image mode: RGB
Image size: (1920, 1080)


IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1920, 1080) to (1920, 1088) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


Image data range: 0 to 255
Frame shape: (1080, 1920, 3)
Frame dtype: uint8

Creating video...
Added frame 1/150
Added frame 31/150
Added frame 61/150
Added frame 91/150
Added frame 121/150
✅ Video created successfully!

✅ Video created: output.mp4
📊 File size: 78.7 KB

Debug steps:
1. Open 'debug_output.png' - does it show your image?
2. Open 'output.mp4' in a video player
3. If still blank, try VLC media player


In [17]:
import nest_asyncio
nest_asyncio.apply()  # Allow nested event loops
import asyncio
from playwright.async_api import async_playwright
import imageio.v2 as iio
import os

async def capture_animated_svg():
    async with async_playwright() as p:
        # Launch browser
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context()
        page = await context.new_page()
        
        # Load the SVG file
        svg_path = os.path.abspath('./github-contribution-grid-snake.svg')
        await page.goto(f'file://{svg_path}')
        
        # Wait for animation to load/start
        await page.wait_for_timeout(1000)
        
        # Set viewport size
        await page.set_viewport_size({'width': 1920, 'height': 1080})
        
        # Capture frames
        fps = 30
        duration = 5  # seconds
        total_frames = fps * duration
        frames = []
        
        print(f"Capturing {total_frames} frames...")
        
        for i in range(total_frames):
            # Take screenshot
            screenshot = await page.screenshot(type='png')
            frames.append(screenshot)
            
            # Wait for next frame (simulate animation)
            await page.wait_for_timeout(1000 // fps)
            
            # Optional: Trigger next animation step if needed
            # await page.evaluate('''() => {
            #     // JavaScript to advance animation if needed
            # }''')
            
            if i % 30 == 0:
                print(f"Captured frame {i+1}/{total_frames}")
        
        await browser.close()
        return frames

# Convert frames to video
async def main():
    frames_data = await capture_animated_svg()
    
    # Create video from frames
    with iio.get_writer('output_animated.mp4', fps=30) as writer:
        for i, frame_data in enumerate(frames_data):
            frame = iio.imread(frame_data)
            writer.append_data(frame)
            if i % 30 == 0:
                print(f"Writing frame {i+1}/{len(frames_data)}")
    
    print("✅ Animated video created: output_animated.mp4")

# Run
asyncio.run(main())

Error: BrowserType.launch: Executable doesn't exist at /home/scientist-anand/.cache/ms-playwright/chromium_headless_shell-1200/chrome-headless-shell-linux64/chrome-headless-shell
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚════════════════════════════════════════════════════════════╝

In [19]:
!playwright install chromium


(node:2080956) [DEP0169] DeprecationWarning: `url.parse()` behavior is not standardized and prone to errors that have security implications. Use the WHATWG URL API instead. CVEs are not issued for `url.parse()` vulnerabilities.
(Use `node --trace-deprecation ...` to show where the warning was created)
164.7 MiB [                    ] 0% 0.0s164.7 MiB [                    ] 0% 211.6s164.7 MiB [                    ] 0% 1406.4s164.7 MiB [                    ] 0% 847.5s164.7 MiB [                    ] 0% 1150.6s164.7 MiB [                    ] 0% 864.2s164.7 MiB [                    ] 0% 1269.4s164.7 MiB [                    ] 0% 1033.1s164.7 MiB [                    ] 0% 1060.0s164.7 MiB [                    ] 0% 907.5s164.7 MiB [                    ] 0% 1227.3s164.7 MiB [                    ] 0% 1076.0s164.7 MiB [                    ] 0% 1087.5s164.7 MiB [                    ] 0% 975.9s164.7 MiB [                    ] 0% 983.2s164.7 MiB [                    ] 0% 898.6s164.7 MiB [        

In [23]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from playwright.async_api import async_playwright
import imageio.v2 as iio
import os
import xml.etree.ElementTree as ET

async def capture_animated_svg_auto():
    """Auto-detect SVG dimensions and capture animation"""
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        
        # Load SVG
        svg_path = os.path.abspath('./github-contribution-grid-snake.svg')
        await page.goto(f'file://{svg_path}')
        
        # Read SVG file to get dimensions
        with open(svg_path, 'r') as f:
            svg_content = f.read()
        
        # Try to parse SVG dimensions
        try:
            root = ET.fromstring(svg_content)
            width = root.get('width', '1920')
            height = root.get('height', '1080')
            viewbox = root.get('viewBox')
            
            if viewbox:
                # Parse viewBox: "min-x min-y width height"
                _, _, vb_width, vb_height = map(float, viewbox.split())
                width = int(vb_width)
                height = int(vb_height)
            else:
                # Convert to integers, removing 'px' if present
                width = int(float(''.join(filter(str.isdigit, width))) or 1920)
                height = int(float(''.join(filter(str.isdigit, height))) or 1080)
            
            print(f"📏 Auto-detected SVG size: {width} x {height}")
        except:
            print("⚠️ Could not auto-detect SVG size, using defaults")
            width, height = 1920, 1080
        
        # Set viewport (add some padding if needed)
        await page.set_viewport_size({'width': width, 'height': height})
        
        # Wait for animation
        await page.wait_for_timeout(2000)
        
        # Capture parameters
        fps = 30
        duration = 40  # seconds
        total_frames = fps * duration
        
        print(f"🎬 Capturing {total_frames} frames ({duration}s @ {fps}fps)...")
        
        # Capture frames
        frames = []
        for i in range(total_frames):
            screenshot = await page.screenshot(type='png')
            frames.append(screenshot)
            
            # Animation progression
            await page.wait_for_timeout(1000 // fps)
            
            # Progress indicator
            if (i + 1) % 30 == 0:
                print(f"  📸 Frame {i + 1}/{total_frames}")
        
        await browser.close()
        
        # Save video
        print("💾 Saving video...")
        with iio.get_writer('output_auto.mp4', fps=fps) as writer:
            for frame_data in frames:
                writer.append_data(iio.imread(frame_data))
        
        print(f"✅ Video saved: output_auto.mp4 ({width}x{height})")

asyncio.run(capture_animated_svg_auto())

📏 Auto-detected SVG size: 880 x 192
🎬 Capturing 1200 frames (40s @ 30fps)...
  📸 Frame 30/1200
  📸 Frame 60/1200
  📸 Frame 90/1200
  📸 Frame 120/1200
  📸 Frame 150/1200
  📸 Frame 180/1200
  📸 Frame 210/1200
  📸 Frame 240/1200
  📸 Frame 270/1200
  📸 Frame 300/1200
  📸 Frame 330/1200
  📸 Frame 360/1200
  📸 Frame 390/1200
  📸 Frame 420/1200
  📸 Frame 450/1200
  📸 Frame 480/1200
  📸 Frame 510/1200
  📸 Frame 540/1200
  📸 Frame 570/1200
  📸 Frame 600/1200
  📸 Frame 630/1200
  📸 Frame 660/1200
  📸 Frame 690/1200
  📸 Frame 720/1200
  📸 Frame 750/1200
  📸 Frame 780/1200
  📸 Frame 810/1200
  📸 Frame 840/1200
  📸 Frame 870/1200
  📸 Frame 900/1200
  📸 Frame 930/1200
  📸 Frame 960/1200
  📸 Frame 990/1200
  📸 Frame 1020/1200
  📸 Frame 1050/1200
  📸 Frame 1080/1200
  📸 Frame 1110/1200
  📸 Frame 1140/1200
  📸 Frame 1170/1200
  📸 Frame 1200/1200
💾 Saving video...
✅ Video saved: output_auto.mp4 (880x192)


In [24]:
import subprocess
import os
from pathlib import Path

def video_to_gif_ffmpeg(input_path, output_path, fps=10, width=640, quality=90, start_time=None, duration=None):
    """
    Convert video to GIF using FFmpeg with full control
    
    Args:
        input_path: Path to input video
        output_path: Path to output GIF
        fps: Frames per second (default: 10)
        width: Output width, height scales automatically (default: 640)
        quality: GIF quality 1-100 (default: 90)
        start_time: Start time in seconds or HH:MM:SS format
        duration: Duration in seconds or HH:MM:SS format
    """
    # Build FFmpeg command
    cmd = ['ffmpeg', '-y']  # -y overwrites output
    
    # Add start time if specified
    if start_time:
        cmd.extend(['-ss', str(start_time)])
    
    # Add input file
    cmd.extend(['-i', input_path])
    
    # Add duration if specified
    if duration:
        cmd.extend(['-t', str(duration)])
    
    # Add filter options
    cmd.extend([
        '-vf', f'fps={fps},scale={width}:-1:flags=lanczos,split[s0][s1];[s0]palettegen=max_colors=256[p];[s1][p]paletteuse',
        '-loop', '0',  # 0 = infinite loop
        output_path
    ])
    
    try:
        # Execute command
        result = subprocess.run(cmd, capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"✅ GIF created successfully: {output_path}")
            
            # Get output file size
            gif_size = os.path.getsize(output_path) / (1024 * 1024)  # MB
            print(f"📁 GIF Size: {gif_size:.2f} MB")
            
            return True
        else:
            print(f"❌ Error creating GIF: {result.stderr}")
            return False
            
    except FileNotFoundError:
        print("❌ FFmpeg not found. Please install FFmpeg:")
        print("   Ubuntu: sudo apt install ffmpeg")
        print("   macOS: brew install ffmpeg")
        print("   Windows: Download from https://ffmpeg.org/")
        return False

# Example usage
input_video = "./output_auto.mp4"
output_gif = "output.gif"

# Convert entire video
video_to_gif_ffmpeg(input_video, output_gif, fps=12, width=800)

# Convert specific segment (10 seconds from 00:30 to 00:40)
video_to_gif_ffmpeg(
    input_video, 
    "clip.gif", 
    fps=15, 
    width=600, 
    start_time="00:30", 
    duration=10
)

✅ GIF created successfully: output.gif
📁 GIF Size: 3.43 MB
✅ GIF created successfully: clip.gif
📁 GIF Size: 0.77 MB


True